In [1]:
# pip install matplotlib

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as sm
import matplotlib.pyplot as plt

### Data Import

In [3]:
# ETF
etf = pd.read_csv('./etf_month_JY.csv')  # % 수익률
etf.columns = ['DATE', 'SPY', 'LQD', 'IEF']
# etf.set_index('DATE', inplace=True)

# Risk Free
rf = pd.read_csv('./rf_JY.csv')
# rf.set_index('DATE', inplace=True)

# Risk Premium
# credit01 = pd.read_csv('./credit01_JY.csv')
# credit02 = pd.read_csv('./credit02_JY.csv')
credit03 = pd.read_csv('./credit03_JY.csv')
# credit03.set_index('DATE', inplace=True)

# term01 = pd.read_csv('./term01_JY.csv')
# term02 = pd.read_csv('./term02_JY.csv')
term03 = pd.read_csv('./term03_JY.csv')
# term03.set_index('DATE', inplace=True)

equity01 = pd.read_csv('./equity01_JY.csv')
# equity01.set_index('DATE', inplace=True)
# equity02 = pd.read_csv('./equity02_JY.csv')

In [4]:
etf.iloc[:,1:] /= 100
etf#.iloc[:,1:].mean()*10000  # 소수점 단위 완료

,DATE,SPY,LQD,IEF
0,2003-01,-0.055013,0.013695,0.008255
1,2003-02,-0.013479,0.019799,0.022040
2,2003-03,0.002140,0.001208,-0.003799
3,2003-04,0.084612,0.023578,0.001380
4,2003-05,0.054836,0.039052,0.037909
...,...,...,...,...
235,2022-08,-0.040802,-0.044291,-0.038538
236,2022-09,-0.092446,-0.059798,-0.047350
237,2022-10,0.081275,-0.007453,-0.014540
238,2022-11,0.055592,0.066477,0.036123


In [5]:
equity01.iloc[:,1] /= 12
equity01  # 연 수익률을 월 환산

,DATE,Return
0,2003-02,-0.020256
1,2003-03,0.005185
2,2003-04,0.077745
3,2003-05,0.047925
4,2003-06,0.008544
...,...,...
222,2021-08,0.027921
223,2021-09,-0.048715
224,2021-10,0.067825
225,2021-11,-0.009633


In [6]:
rf.iloc[:,1] /= 12
rf  # 연 수익률을 월 환산

,DATE,RF
0,2003-01,0.008333
1,2003-02,0.007500
2,2003-03,0.008333
3,2003-04,0.008333
4,2003-05,0.007500
...,...,...
221,2021-06,0.000000
222,2021-07,0.000000
223,2021-08,0.000000
224,2021-09,0.000000


In [7]:
credit03.iloc[:,1] /= 12
credit03

,DATE,Return
0,2003-01,0.004146
1,2003-02,0.003973
2,2003-03,0.003951
3,2003-04,0.003852
4,2003-05,0.003472
...,...,...
223,2021-08,0.002080
224,2021-09,0.002050
225,2021-10,0.002188
226,2021-11,0.002145


In [8]:
term03.iloc[:,1] /= 12
term03

,DATE,Return
0,2003-01,0.002383
1,2003-02,0.002264
2,2003-03,0.002213
3,2003-04,0.002338
4,2003-05,0.002067
...,...,...
223,2021-08,0.001024
224,2021-09,0.001111
225,2021-10,0.001275
226,2021-11,0.001257


In [9]:
# Regime : List
recovery = ['2003-04', '2003-05', '2003-06', '2003-07', '2003-08', '2003-09', '2009-05', '2009-06', '2009-07', '2009-08', '2009-09', '2009-10', '2009-11', '2009-12', '2010-01', '2010-02', '2010-03', '2010-04', '2010-05', '2010-07', '2010-08', '2010-09', '2010-10', '2011-06', '2011-12', '2012-01', '2012-06', '2012-07', '2012-08', '2012-09', '2012-10', '2012-11', '2015-10', '2016-03', '2016-04', '2016-05', '2016-06', '2016-07', '2016-08', '2016-09', '2016-11', '2016-12', '2017-01', '2017-02', '2019-02', '2019-03', '2019-04', '2019-06', '2019-09', '2019-10', '2019-11', '2019-12', '2020-11']
expansion = ['2003-10', '2003-11', '2003-12', '2004-01', '2004-02', '2004-03', '2004-04', '2004-05', '2004-06', '2004-07', '2004-08', '2004-09', '2004-10', '2004-11', '2004-12', '2005-01', '2005-02', '2005-03', '2005-04', '2005-05', '2005-06', '2005-07', '2005-08', '2005-09', '2005-11', '2005-12', '2006-01', '2006-02', '2006-03', '2006-04', '2006-08', '2006-09', '2006-10', '2006-11', '2006-12', '2007-01', '2008-05', '2010-11', '2010-12', '2011-01', '2011-02', '2011-03', '2011-04', '2011-05', '2012-02', '2012-03', '2012-04', '2012-12', '2013-01', '2013-02', '2013-03', '2013-04', '2013-05', '2013-06', '2013-07', '2013-09', '2013-10', '2013-11', '2013-12', '2014-02', '2014-03', '2014-04', '2014-05', '2014-06', '2014-08', '2014-10', '2014-11', '2015-02', '2015-04', '2015-05', '2015-07', '2017-03', '2017-04', '2017-05', '2017-06', '2017-07', '2017-08', '2017-09', '2017-10', '2017-11', '2017-12', '2018-07', '2018-08', '2018-09', '2020-12', '2021-03', '2021-04', '2021-05', '2021-06', '2021-07', '2021-08', '2021-09', '2021-10']
slowdown = ['2005-10', '2006-05', '2006-06', '2006-07', '2007-02', '2007-03', '2007-04', '2007-05', '2007-06', '2007-07', '2007-08', '2007-09', '2007-10', '2007-11', '2007-12', '2008-01', '2008-02', '2008-03', '2008-04', '2008-06', '2013-08', '2014-01', '2014-07', '2014-09', '2014-12', '2015-01', '2015-03', '2015-06', '2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06', '2018-10', '2018-11', '2018-12', '2021-01', '2021-02']
contraction = ['2003-01', '2003-02', '2003-03', '2008-07', '2008-08', '2008-09', '2008-10', '2008-11', '2008-12', '2009-01', '2009-02', '2009-03', '2009-04', '2010-06', '2011-07', '2011-08', '2011-09', '2011-10', '2011-11', '2012-05', '2015-08', '2015-09', '2015-11', '2015-12', '2016-01', '2016-02', '2016-10', '2019-01', '2019-05', '2019-07', '2019-08', '2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06', '2020-07', '2020-08', '2020-09', '2020-10']

In [10]:
# Buy and Hold : List


start_date = '2003-01-01'
end_date = '2021-10-31'

bh = pd.date_range(start=start_date, end=end_date, freq='M').strftime('%Y-%m-%d').tolist()

In [11]:
bh

['2003-01-31',
 '2003-02-28',
 '2003-03-31',
 '2003-04-30',
 '2003-05-31',
 '2003-06-30',
 '2003-07-31',
 '2003-08-31',
 '2003-09-30',
 '2003-10-31',
 '2003-11-30',
 '2003-12-31',
 '2004-01-31',
 '2004-02-29',
 '2004-03-31',
 '2004-04-30',
 '2004-05-31',
 '2004-06-30',
 '2004-07-31',
 '2004-08-31',
 '2004-09-30',
 '2004-10-31',
 '2004-11-30',
 '2004-12-31',
 '2005-01-31',
 '2005-02-28',
 '2005-03-31',
 '2005-04-30',
 '2005-05-31',
 '2005-06-30',
 '2005-07-31',
 '2005-08-31',
 '2005-09-30',
 '2005-10-31',
 '2005-11-30',
 '2005-12-31',
 '2006-01-31',
 '2006-02-28',
 '2006-03-31',
 '2006-04-30',
 '2006-05-31',
 '2006-06-30',
 '2006-07-31',
 '2006-08-31',
 '2006-09-30',
 '2006-10-31',
 '2006-11-30',
 '2006-12-31',
 '2007-01-31',
 '2007-02-28',
 '2007-03-31',
 '2007-04-30',
 '2007-05-31',
 '2007-06-30',
 '2007-07-31',
 '2007-08-31',
 '2007-09-30',
 '2007-10-31',
 '2007-11-30',
 '2007-12-31',
 '2008-01-31',
 '2008-02-29',
 '2008-03-31',
 '2008-04-30',
 '2008-05-31',
 '2008-06-30',
 '2008-07-

### Recovery - SPY

In [12]:
# 데이터 병합
etf_month_spy = etf[['DATE', 'SPY']]
data01 = etf_month_spy.merge(rf, on='DATE', how='inner')
# data01 = data01.merge(credit01, on='DATE', how='inner', suffixes=('', '_credit01'))
# data01 = data01.merge(credit02, on='DATE', how='inner', suffixes=('', '_credit02'))
data01 = data01.merge(credit03, on='DATE', how='inner', suffixes=('', '_credit03'))
# data01 = data01.merge(term01, on='DATE', how='inner', suffixes=('', '_term01'))
# data01 = data01.merge(term02, on='DATE', how='inner', suffixes=('', '_term02'))
data01 = data01.merge(term03, on='DATE', how='inner', suffixes=('', '_term03'))
data01 = data01.merge(equity01, on='DATE', how='inner', suffixes=('', '_equity01'))
# data01 = data01.merge(equity02, on='DATE', how='inner', suffixes=('', '_equity02'))

# Renaming columns as requested
data01.columns = ['DATE', 'SPY', 'rf', 'credit03', 'term03', 'equity01']
data01['DATE'] = pd.to_datetime(data01['DATE'])
data01['DATE'] = data01['DATE'].dt.strftime('%Y-%m')
data01.set_index('DATE', inplace=True)

# Assuming recovery_dates is a list of datetime objects representing the recovery dates
recovery_dates = [pd.to_datetime(date).strftime('%Y-%m') for date in recovery] 
recovery_dates

# recovery 컬럼 추가
data01['recovery'] = data01.index.map(lambda x: 1 if x in recovery_dates else 0)
# data01['recovery'].unique()

# recovery 국면만 슬라이싱
data01 = data01[data01['recovery'] == 1]
data01

# Selecting relevant columns for the regression
X = data01[['credit03', 'term03', 'equity01']]
y = data01['SPY']-data01['rf']

# # Adding a constant to the independent variables
X = sm.add_constant(X)

# Performing the OLS regression
model01 = sm.OLS(y, X, missing='drop').fit()

# Displaying the regression results
model01.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.989
Model:                            OLS   Adj. R-squared (uncentered):              0.988
Method:                 Least Squares   F-statistic:                              1484.
Date:                Fri, 24 Nov 2023   Prob (F-statistic):                    7.87e-49
Time:                        22:06:47   Log-Likelihood:                          212.58
No. Observations:                  53   AIC:                                     -419.2
Df Residuals:                      50   BIC:                                     -413.2
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
credit03      -3.2771      1.093     -2.997      0.004      -5.473      -1.081
term03         6.5151      1.752      3.718      0.001       2.995      10.035
equity01       0.9916      0.018     54.130      0.000       0.955       1.028
==============================================================================
Omnibus:                        4.254   Durbin-Watson:                   0.710
Prob(Omnibus):                  0.119   Jarque-Bera (JB):                3.973
Skew:                          -0.607   Prob(JB):                        0.137
Kurtosis:                       2.431   Cond. No.                         138.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
data01

,SPY,rf,credit03,term03,equity01,recovery
DATE,,,,,,
2003-04,0.084612,0.008333,0.003852,0.002338,0.077745,1
2003-05,0.054836,0.007500,0.003472,0.002067,0.047925,1
2003-06,0.010652,0.008333,0.003351,0.001998,0.008544,1
2003-07,0.018028,0.005833,0.003727,0.002546,0.012911,1
2003-08,0.020626,0.005833,0.004086,0.002897,0.014169,1
2003-09,-0.010906,0.006667,0.003994,0.002766,-0.015506,1
2009-05,0.058453,0.000000,0.004463,0.002597,0.050337,1
2009-06,-0.000655,0.000833,0.004543,0.002953,-0.002906,1
2009-07,0.074606,0.000833,0.004354,0.002816,0.071173,1


### Recovery - LQD

In [14]:
# 데이터 병합

etf_month_LQD = etf[['DATE', 'LQD']]
data02 = etf_month_LQD.merge(rf, on='DATE', how='inner')
# data02 = data02.merge(credit01, on='DATE', how='inner', suffixes=('', '_credit01'))
# data02 = data02.merge(credit02, on='DATE', how='inner', suffixes=('', '_credit02'))
data02 = data02.merge(credit03, on='DATE', how='inner', suffixes=('', '_credit03'))
# data02 = data02.merge(term01, on='DATE', how='inner', suffixes=('', '_term01'))
# data02 = data02.merge(term02, on='DATE', how='inner', suffixes=('', '_term02'))
data02 = data02.merge(term03, on='DATE', how='inner', suffixes=('', '_term03'))
data02 = data02.merge(equity01, on='DATE', how='inner', suffixes=('', '_equity01'))
# data02 = data02.merge(equity02, on='DATE', how='inner', suffixes=('', '_equity02'))

# Renaming columns as requested
data02.columns = ['DATE', 'LQD', 'rf', 'credit03', 'term03', 'equity01']
data02['DATE'] = pd.to_datetime(data02['DATE'])
data02['DATE'] = data02['DATE'].dt.strftime('%Y-%m')
data02.set_index('DATE', inplace=True)

# Assuming recovery_dates is a list of datetime objects representing the recovery dates
recovery_dates = [pd.to_datetime(date).strftime('%Y-%m') for date in recovery] 
recovery_dates

# recovery 컬럼 추가
data02['recovery'] = data02.index.map(lambda x: 1 if x in recovery_dates else 0)

# recovery 국면만 슬라이싱
data02 = data02[data02['recovery'] == 1]
data02

# Selecting relevant columns for the regression
X = data02[['credit03', 'term03', 'equity01']]
y = data02['LQD']-data02['rf']

# # Adding a constant to the independent variables
X = sm.add_constant(X)

# Performing the OLS regression
model02 = sm.OLS(y, X, missing='drop').fit()

# Displaying the regression results
model02.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.190
Model:                            OLS   Adj. R-squared (uncentered):              0.142
Method:                 Least Squares   F-statistic:                              3.918
Date:                Fri, 24 Nov 2023   Prob (F-statistic):                      0.0137
Time:                        22:06:47   Log-Likelihood:                          134.86
No. Observations:                  53   AIC:                                     -263.7
Df Residuals:                      50   BIC:                                     -257.8
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
credit03       0.8361      4.738      0.176      0.861      -8.680      10.353
term03         1.5624      7.594      0.206      0.838     -13.690      16.815
equity01       0.1204      0.079      1.517      0.136      -0.039       0.280
==============================================================================
Omnibus:                       13.000   Durbin-Watson:                   1.906
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               17.860
Skew:                          -0.828   Prob(JB):                     0.000132
Kurtosis:                       5.312   Cond. No.                         138.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Recovery - IEF

In [15]:
# 데이터 병합
etf_month_IEF = etf[['DATE', 'IEF']]
data03 = etf_month_IEF.merge(rf, on='DATE', how='inner')
# data03 = data03.merge(credit01, on='DATE', how='inner', suffixes=('', '_credit01'))
# data03 = data03.merge(credit02, on='DATE', how='inner', suffixes=('', '_credit02'))
data03 = data03.merge(credit03, on='DATE', how='inner', suffixes=('', '_credit03'))
# data03 = data03.merge(term01, on='DATE', how='inner', suffixes=('', '_term01'))
# data03 = data03.merge(term02, on='DATE', how='inner', suffixes=('', '_term02'))
data03 = data03.merge(term03, on='DATE', how='inner', suffixes=('', '_term03'))
data03 = data03.merge(equity01, on='DATE', how='inner', suffixes=('', '_equity01'))
# data03 = data03.merge(equity02, on='DATE', how='inner', suffixes=('', '_equity02'))

# Renaming columns as requested
data03.columns = ['DATE', 'IEF', 'rf', 'credit03', 'term03', 'equity01']
data03['DATE'] = pd.to_datetime(data03['DATE'])
data03['DATE'] = data03['DATE'].dt.strftime('%Y-%m')
data03.set_index('DATE', inplace=True)

# Assuming recovery_dates is a list of datetime objects representing the recovery dates
recovery_dates = [pd.to_datetime(date).strftime('%Y-%m') for date in recovery] 
recovery_dates

# recovery 컬럼 추가
data03['recovery'] = data03.index.map(lambda x: 1 if x in recovery_dates else 0)

# recovery 국면만 슬라이싱
data03 = data03[data03['recovery'] == 1]
data03

# Selecting relevant columns for the regression
X = data03[['credit03', 'term03', 'equity01']]
y = data03['IEF'] - data03['rf']

# Adding a constant to the independent variables
import statsmodels.api as sm
X = sm.add_constant(X)

# Performing the OLS regression
model03 = sm.OLS(y, X, missing='drop').fit()

# Displaying the regression results
model03.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.091
Model:                            OLS   Adj. R-squared (uncentered):              0.036
Method:                 Least Squares   F-statistic:                              1.662
Date:                Fri, 24 Nov 2023   Prob (F-statistic):                       0.187
Time:                        22:06:47   Log-Likelihood:                          138.10
No. Observations:                  53   AIC:                                     -270.2
Df Residuals:                      50   BIC:                                     -264.3
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
credit03      -2.2878      4.457     -0.513      0.610     -11.239       6.663
term03         5.5488      7.143      0.777      0.441      -8.798      19.896
equity01      -0.1239      0.075     -1.659      0.103      -0.274       0.026
==============================================================================
Omnibus:                       12.359   Durbin-Watson:                   2.202
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               14.487
Skew:                          -0.889   Prob(JB):                     0.000715
Kurtosis:                       4.844   Cond. No.                         138.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Expansion - SPY

In [16]:
# 데이터 병합
etf_month_spy = etf[['DATE', 'SPY']]
data04 = etf_month_spy.merge(rf, on='DATE', how='inner')
# data04 = data04.merge(credit01, on='DATE', how='inner', suffixes=('', '_credit01'))
# data04 = data04.merge(credit02, on='DATE', how='inner', suffixes=('', '_credit02'))
data04 = data04.merge(credit03, on='DATE', how='inner', suffixes=('', '_credit03'))
# data04 = data04.merge(term01, on='DATE', how='inner', suffixes=('', '_term01'))
# data04 = data04.merge(term02, on='DATE', how='inner', suffixes=('', '_term02'))
data04 = data04.merge(term03, on='DATE', how='inner', suffixes=('', '_term03'))
data04 = data04.merge(equity01, on='DATE', how='inner', suffixes=('', '_equity01'))
# data04 = data04.merge(equity02, on='DATE', how='inner', suffixes=('', '_equity02'))

# Renaming columns as requested
data04.columns = ['DATE', 'SPY', 'rf', 'credit03','term03', 'equity01']
data04['DATE'] = pd.to_datetime(data04['DATE'])
data04['DATE'] = data04['DATE'].dt.strftime('%Y-%m')
data04.set_index('DATE', inplace=True)

# Assuming expansion_dates is a list of datetime objects representing the expansion dates
expansion_dates = [pd.to_datetime(date).strftime('%Y-%m') for date in expansion] 
expansion_dates

# expansion 컬럼 추가
data04['expansion'] = data04.index.map(lambda x: 1 if x in expansion_dates else 0)

# expansion 국면만 슬라이싱
data04 = data04[data04['expansion'] == 1]
data04

# Selecting relevant columns for the regression
X = data04[['credit03', 'term03', 'equity01']]
y = data04['SPY'] - data04['rf']

# Adding a constant to the independent variables
import statsmodels.api as sm
X = sm.add_constant(X)

# Performing the OLS regression
model04 = sm.OLS(y, X, missing='drop').fit()

# Displaying the regression results
model04.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.876
Model:                            OLS   Adj. R-squared (uncentered):              0.872
Method:                 Least Squares   F-statistic:                              211.7
Date:                Fri, 24 Nov 2023   Prob (F-statistic):                    1.19e-40
Time:                        22:06:47   Log-Likelihood:                          300.29
No. Observations:                  93   AIC:                                     -594.6
Df Residuals:                      90   BIC:                                     -587.0
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
credit03     -12.1858      2.365     -5.154      0.000     -16.883      -7.488
term03        19.8477      3.797      5.227      0.000      12.304      27.391
equity01       1.0282      0.048     21.213      0.000       0.932       1.125
==============================================================================
Omnibus:                       13.522   Durbin-Watson:                   0.216
Prob(Omnibus):                  0.001   Jarque-Bera (JB):                7.042
Skew:                          -0.484   Prob(JB):                       0.0296
Kurtosis:                       2.062   Cond. No.                         115.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Expansion - LQD

In [17]:
# 데이터 병합
etf_month_LQD = etf[['DATE', 'LQD']]
data05 = etf_month_LQD.merge(rf, on='DATE', how='inner')
# data05 = data05.merge(credit01, on='DATE', how='inner', suffixes=('', '_credit01'))
# data05 = data05.merge(credit02, on='DATE', how='inner', suffixes=('', '_credit02'))
data05 = data05.merge(credit03, on='DATE', how='inner', suffixes=('', '_credit03'))
# data05 = data05.merge(term01, on='DATE', how='inner', suffixes=('', '_term01'))
# data05 = data05.merge(term02, on='DATE', how='inner', suffixes=('', '_term02'))
data05 = data05.merge(term03, on='DATE', how='inner', suffixes=('', '_term03'))
data05 = data05.merge(equity01, on='DATE', how='inner', suffixes=('', '_equity01'))
# data05 = data05.merge(equity02, on='DATE', how='inner', suffixes=('', '_equity02'))

# Renaming columns as requested
data05.columns = ['DATE', 'LQD', 'rf', 'credit03', 'term03', 'equity01']
data05['DATE'] = pd.to_datetime(data05['DATE'])
data05['DATE'] = data05['DATE'].dt.strftime('%Y-%m')
data05.set_index('DATE', inplace=True)

# Assuming expansion_dates is a list of datetime objects representing the expansion dates
expansion_dates = [pd.to_datetime(date).strftime('%Y-%m') for date in expansion] 
expansion_dates

# expansion 컬럼 추가
data05['expansion'] = data05.index.map(lambda x: 1 if x in expansion_dates else 0)

# expansion 국면만 슬라이싱
data05 = data05[data05['expansion'] == 1]
data05

# Selecting relevant columns for the regression
X = data05[['credit03', 'term03', 'equity01']]
y = data05['LQD'] - data05['rf']

# Adding a constant to the independent variables
import statsmodels.api as sm
X = sm.add_constant(X)

# Performing the OLS regression
model05 = sm.OLS(y, X, missing='drop').fit()

# Displaying the regression results
model05.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.149
Model:                            OLS   Adj. R-squared (uncentered):              0.121
Method:                 Least Squares   F-statistic:                              5.272
Date:                Fri, 24 Nov 2023   Prob (F-statistic):                     0.00214
Time:                        22:06:47   Log-Likelihood:                          250.22
No. Observations:                  93   AIC:                                     -494.4
Df Residuals:                      90   BIC:                                     -486.8
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
credit03     -15.9977      4.051     -3.949      0.000     -24.045      -7.950
term03        25.0030      6.505      3.844      0.000      12.081      37.925
equity01       0.0991      0.083      1.193      0.236      -0.066       0.264
==============================================================================
Omnibus:                        3.393   Durbin-Watson:                   1.206
Prob(Omnibus):                  0.183   Jarque-Bera (JB):                3.189
Skew:                          -0.452   Prob(JB):                        0.203
Kurtosis:                       2.933   Cond. No.                         115.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Expansion - IEF

In [18]:
# 데이터 병합
etf_month_IEF = etf[['DATE', 'IEF']]
data06 = etf_month_IEF.merge(rf, on='DATE', how='inner')
# data06 = data06.merge(credit01, on='DATE', how='inner', suffixes=('', '_credit01'))
# data06 = data06.merge(credit02, on='DATE', how='inner', suffixes=('', '_credit02'))
data06 = data06.merge(credit03, on='DATE', how='inner', suffixes=('', '_credit03'))
# data06 = data06.merge(term01, on='DATE', how='inner', suffixes=('', '_term01'))
# data06 = data06.merge(term02, on='DATE', how='inner', suffixes=('', '_term02'))
data06 = data06.merge(term03, on='DATE', how='inner', suffixes=('', '_term03'))
data06 = data06.merge(equity01, on='DATE', how='inner', suffixes=('', '_equity01'))
# data06 = data06.merge(equity02, on='DATE', how='inner', suffixes=('', '_equity02'))

# Renaming columns as requested
data06.columns = ['DATE', 'IEF', 'rf', 'credit03', 'term03', 'equity01']
data06['DATE'] = pd.to_datetime(data06['DATE'])
data06['DATE'] = data06['DATE'].dt.strftime('%Y-%m')
data06.set_index('DATE', inplace=True)

# Assuming expansion_dates is a list of datetime objects representing the expansion dates
expansion_dates = [pd.to_datetime(date).strftime('%Y-%m') for date in expansion] 
expansion_dates

# expansion 컬럼 추가
data06['expansion'] = data06.index.map(lambda x: 1 if x in expansion_dates else 0)

# expansion 국면만 슬라이싱
data06 = data06[data06['expansion'] == 1]
data06

# Selecting relevant columns for the regression
X = data06[['credit03', 'term03', 'equity01']]
y = data06['IEF'] - data06['rf']

# Adding a constant to the independent variables
import statsmodels.api as sm
X = sm.add_constant(X)

# Performing the OLS regression
model06 = sm.OLS(y, X, missing='drop').fit()

# Displaying the regression results
model06.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.174
Model:                            OLS   Adj. R-squared (uncentered):              0.146
Method:                 Least Squares   F-statistic:                              6.319
Date:                Fri, 24 Nov 2023   Prob (F-statistic):                    0.000614
Time:                        22:06:47   Log-Likelihood:                          243.84
No. Observations:                  93   AIC:                                     -481.7
Df Residuals:                      90   BIC:                                     -474.1
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
credit03     -15.4269      4.338     -3.556      0.001     -24.046      -6.808
term03        24.1353      6.966      3.464      0.001      10.295      37.975
equity01      -0.0585      0.089     -0.657      0.513      -0.235       0.118
==============================================================================
Omnibus:                        2.676   Durbin-Watson:                   1.303
Prob(Omnibus):                  0.262   Jarque-Bera (JB):                2.251
Skew:                          -0.378   Prob(JB):                        0.325
Kurtosis:                       3.096   Cond. No.                         115.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Slowdown - SPY

In [19]:
# 데이터 병합
etf_month_spy = etf[['DATE', 'SPY']]
data07 = etf_month_spy.merge(rf, on='DATE', how='inner')
# data07 = data07.merge(credit01, on='DATE', how='inner', suffixes=('', '_credit01'))
# data07 = data07.merge(credit02, on='DATE', how='inner', suffixes=('', '_credit02'))
data07 = data07.merge(credit03, on='DATE', how='inner', suffixes=('', '_credit03'))
# data07 = data07.merge(term01, on='DATE', how='inner', suffixes=('', '_term01'))
# data07 = data07.merge(term02, on='DATE', how='inner', suffixes=('', '_term02'))
data07 = data07.merge(term03, on='DATE', how='inner', suffixes=('', '_term03'))
data07 = data07.merge(equity01, on='DATE', how='inner', suffixes=('', '_equity01'))
# data07 = data07.merge(equity02, on='DATE', how='inner', suffixes=('', '_equity02'))

# Renaming columns as requested
data07.columns = ['DATE', 'SPY', 'rf', 'credit03', 'term03', 'equity01']
data07['DATE'] = pd.to_datetime(data07['DATE'])
data07['DATE'] = data07['DATE'].dt.strftime('%Y-%m')
data07.set_index('DATE', inplace=True)

# Assuming slowdown_dates is a list of datetime objects representing the slowdown dates
slowdown_dates = [pd.to_datetime(date).strftime('%Y-%m') for date in slowdown] 
slowdown_dates

# slowdown 컬럼 추가
data07['slowdown'] = data07.index.map(lambda x: 1 if x in slowdown_dates else 0)

# slowdown 국면만 슬라이싱
data07 = data07[data07['slowdown'] == 1]
data07

# Selecting relevant columns for the regression
X = data07[['credit03', 'term03', 'equity01']]
y = data07['SPY'] - data07['rf']

# Adding a constant to the independent variables
import statsmodels.api as sm
X = sm.add_constant(X)

# Performing the OLS regression
model07 = sm.OLS(y, X, missing='drop').fit()

# Displaying the regression results
model07.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.961
Model:                            OLS   Adj. R-squared (uncentered):              0.958
Method:                 Least Squares   F-statistic:                              296.8
Date:                Fri, 24 Nov 2023   Prob (F-statistic):                    1.95e-25
Time:                        22:06:48   Log-Likelihood:                          130.75
No. Observations:                  39   AIC:                                     -255.5
Df Residuals:                      36   BIC:                                     -250.5
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
credit03     -32.5309      3.022    -10.765      0.000     -38.660     -26.402
term03        56.2099      5.547     10.134      0.000      44.960      67.459
equity01       0.9849      0.043     23.048      0.000       0.898       1.072
==============================================================================
Omnibus:                        3.223   Durbin-Watson:                   0.642
Prob(Omnibus):                  0.200   Jarque-Bera (JB):                2.606
Skew:                           0.633   Prob(JB):                        0.272
Kurtosis:                       2.977   Cond. No.                         163.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Slowdown - LQD

In [20]:
# 데이터 병합
etf_month_LQD = etf[['DATE', 'LQD']]
data08 = etf_month_LQD.merge(rf, on='DATE', how='inner')
# data08 = data08.merge(credit01, on='DATE', how='inner', suffixes=('', '_credit01'))
# data08 = data08.merge(credit02, on='DATE', how='inner', suffixes=('', '_credit02'))
data08 = data08.merge(credit03, on='DATE', how='inner', suffixes=('', '_credit03'))
# data08 = data08.merge(term01, on='DATE', how='inner', suffixes=('', '_term01'))
# data08 = data08.merge(term02, on='DATE', how='inner', suffixes=('', '_term02'))
data08 = data08.merge(term03, on='DATE', how='inner', suffixes=('', '_term03'))
data08 = data08.merge(equity01, on='DATE', how='inner', suffixes=('', '_equity01'))
# data08 = data08.merge(equity02, on='DATE', how='inner', suffixes=('', '_equity02'))

# Renaming columns as requested
data08.columns = ['DATE', 'LQD', 'rf', 'credit03', 'term03', 'equity01']
data08['DATE'] = pd.to_datetime(data08['DATE'])
data08['DATE'] = data08['DATE'].dt.strftime('%Y-%m')
data08.set_index('DATE', inplace=True)

# Assuming slowdown_dates is a list of datetime objects representing the slowdown dates
slowdown_dates = [pd.to_datetime(date).strftime('%Y-%m') for date in slowdown] 
slowdown_dates

# slowdown 컬럼 추가
data08['slowdown'] = data08.index.map(lambda x: 1 if x in slowdown_dates else 0)

# slowdown 국면만 슬라이싱
data08 = data08[data08['slowdown'] == 1]
data08

# Selecting relevant columns for the regression
X = data08[['credit03', 'term03', 'equity01']]
y = data08['LQD'] - data08['rf']

# Adding a constant to the independent variables
import statsmodels.api as sm
X = sm.add_constant(X)

# Performing the OLS regression
model08 = sm.OLS(y, X, missing='drop').fit()

# Displaying the regression results
model08.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.500
Model:                            OLS   Adj. R-squared (uncentered):              0.459
Method:                 Least Squares   F-statistic:                              12.01
Date:                Fri, 24 Nov 2023   Prob (F-statistic):                    1.34e-05
Time:                        22:06:48   Log-Likelihood:                          102.40
No. Observations:                  39   AIC:                                     -198.8
Df Residuals:                      36   BIC:                                     -193.8
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
credit03     -33.5987      6.252     -5.374      0.000     -46.277     -20.920
term03        54.0369     11.475      4.709      0.000      30.765      77.309
equity01      -0.0256      0.088     -0.290      0.774      -0.205       0.154
==============================================================================
Omnibus:                        5.083   Durbin-Watson:                   1.540
Prob(Omnibus):                  0.079   Jarque-Bera (JB):                3.703
Skew:                           0.669   Prob(JB):                        0.157
Kurtosis:                       3.699   Cond. No.                         163.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Slowdown - IEF

In [21]:
# 데이터 병합
etf_month_IEF = etf[['DATE', 'IEF']]
data09 = etf_month_IEF.merge(rf, on='DATE', how='inner')
# data09 = data09.merge(credit01, on='DATE', how='inner', suffixes=('', '_credit01'))
# data09 = data09.merge(credit02, on='DATE', how='inner', suffixes=('', '_credit02'))
data09 = data09.merge(credit03, on='DATE', how='inner', suffixes=('', '_credit03'))
# data09 = data09.merge(term01, on='DATE', how='inner', suffixes=('', '_term01'))
# data09 = data09.merge(term02, on='DATE', how='inner', suffixes=('', '_term02'))
data09 = data09.merge(term03, on='DATE', how='inner', suffixes=('', '_term03'))
data09 = data09.merge(equity01, on='DATE', how='inner', suffixes=('', '_equity01'))
# data09 = data09.merge(equity02, on='DATE', how='inner', suffixes=('', '_equity02'))

# Renaming columns as requested
data09.columns = ['DATE', 'IEF', 'rf', 'credit03', 'term03', 'equity01']
data09['DATE'] = pd.to_datetime(data09['DATE'])
data09['DATE'] = data09['DATE'].dt.strftime('%Y-%m')
data09.set_index('DATE', inplace=True)

# Assuming slowdown_dates is a list of datetime objects representing the slowdown dates
slowdown_dates = [pd.to_datetime(date).strftime('%Y-%m') for date in slowdown] 
slowdown_dates

# slowdown 컬럼 추가
data09['slowdown'] = data09.index.map(lambda x: 1 if x in slowdown_dates else 0)

# slowdown 국면만 슬라이싱
data09 = data09[data09['slowdown'] == 1]
data09

# Selecting relevant columns for the regression
X = data09[['credit03', 'term03', 'equity01']]
y = data09['IEF'] - data09['rf']

# Adding a constant to the independent variables
import statsmodels.api as sm
X = sm.add_constant(X)

# Performing the OLS regression
model09 = sm.OLS(y, X, missing='drop').fit()

# Displaying the regression results
model09.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.425
Model:                            OLS   Adj. R-squared (uncentered):              0.377
Method:                 Least Squares   F-statistic:                              8.868
Date:                Fri, 24 Nov 2023   Prob (F-statistic):                    0.000156
Time:                        22:06:48   Log-Likelihood:                          102.88
No. Observations:                  39   AIC:                                     -199.8
Df Residuals:                      36   BIC:                                     -194.8
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
credit03     -27.7254      6.175     -4.490      0.000     -40.250     -15.201
term03        44.0039     11.335      3.882      0.000      21.016      66.992
equity01      -0.2454      0.087     -2.810      0.008      -0.422      -0.068
==============================================================================
Omnibus:                        1.127   Durbin-Watson:                   1.422
Prob(Omnibus):                  0.569   Jarque-Bera (JB):                0.639
Skew:                           0.311   Prob(JB):                        0.726
Kurtosis:                       3.077   Cond. No.                         163.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Contraction - SPY

In [22]:
# 데이터 병합
etf_month_spy = etf[['DATE', 'SPY']]
data10 = etf_month_spy.merge(rf, on='DATE', how='inner')
# data10 = data10.merge(credit01, on='DATE', how='inner', suffixes=('', '_credit01'))
# data10 = data10.merge(credit02, on='DATE', how='inner', suffixes=('', '_credit02'))
data10 = data10.merge(credit03, on='DATE', how='inner', suffixes=('', '_credit03'))
# data10 = data10.merge(term01, on='DATE', how='inner', suffixes=('', '_term01'))
# data10 = data10.merge(term02, on='DATE', how='inner', suffixes=('', '_term02'))
data10 = data10.merge(term03, on='DATE', how='inner', suffixes=('', '_term03'))
data10 = data10.merge(equity01, on='DATE', how='inner', suffixes=('', '_equity01'))
# data10 = data10.merge(equity02, on='DATE', how='inner', suffixes=('', '_equity02'))

# Renaming columns as requested
data10.columns = ['DATE', 'SPY', 'rf', 'credit03', 'term03', 'equity01']
data10['DATE'] = pd.to_datetime(data10['DATE'])
data10['DATE'] = data10['DATE'].dt.strftime('%Y-%m')
data10.set_index('DATE', inplace=True)

# Assuming contraction_dates is a list of datetime objects representing the contraction dates
contraction_dates = [pd.to_datetime(date).strftime('%Y-%m') for date in contraction] 
contraction_dates

# contraction 컬럼 추가
data10['contraction'] = data10.index.map(lambda x: 1 if x in contraction_dates else 0)

# contraction 국면만 슬라이싱
data10 = data10[data10['contraction'] == 1]
data10

# Selecting relevant columns for the regression
X = data10[['credit03', 'term03', 'equity01']]
y = data10['SPY'] - data10['rf']

# Adding a constant to the independent variables
import statsmodels.api as sm
X = sm.add_constant(X)

# Performing the OLS regression
model10 = sm.OLS(y, X, missing='drop').fit()

# Displaying the regression results
model10.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.993
Model:                            OLS   Adj. R-squared (uncentered):              0.992
Method:                 Least Squares   F-statistic:                              1762.
Date:                Fri, 24 Nov 2023   Prob (F-statistic):                    5.92e-40
Time:                        22:06:48   Log-Likelihood:                          150.34
No. Observations:                  40   AIC:                                     -294.7
Df Residuals:                      37   BIC:                                     -289.6
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
credit03      -3.2313      1.360     -2.375      0.023      -5.988      -0.475
term03         6.8423      2.658      2.574      0.014       1.457      12.227
equity01       1.0115      0.015     68.545      0.000       0.982       1.041
==============================================================================
Omnibus:                        8.024   Durbin-Watson:                   0.804
Prob(Omnibus):                  0.018   Jarque-Bera (JB):                7.841
Skew:                          -1.084   Prob(JB):                       0.0198
Kurtosis:                       3.045   Cond. No.                         215.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Contraction - LQD

In [23]:
# 데이터 병합
etf_month_LQD = etf[['DATE', 'LQD']]
data11 = etf_month_LQD.merge(rf, on='DATE', how='inner')
# data11 = data11.merge(credit01, on='DATE', how='inner', suffixes=('', '_credit01'))
# data11 = data11.merge(credit02, on='DATE', how='inner', suffixes=('', '_credit02'))
data11 = data11.merge(credit03, on='DATE', how='inner', suffixes=('', '_credit03'))
# data11 = data11.merge(term01, on='DATE', how='inner', suffixes=('', '_term01'))
# data11 = data11.merge(term02, on='DATE', how='inner', suffixes=('', '_term02'))
data11 = data11.merge(term03, on='DATE', how='inner', suffixes=('', '_term03'))
data11 = data11.merge(equity01, on='DATE', how='inner', suffixes=('', '_equity01'))
# data11 = data11.merge(equity02, on='DATE', how='inner', suffixes=('', '_equity02'))

# Renaming columns as requested
data11.columns = ['DATE', 'LQD', 'rf', 'credit03', 'term03', 'equity01']
data11['DATE'] = pd.to_datetime(data11['DATE'])
data11['DATE'] = data11['DATE'].dt.strftime('%Y-%m')
data11.set_index('DATE', inplace=True)

# Assuming contraction_dates is a list of datetime objects representing the contraction dates
contraction_dates = [pd.to_datetime(date).strftime('%Y-%m') for date in contraction] 
contraction_dates

# contraction 컬럼 추가
data11['contraction'] = data11.index.map(lambda x: 1 if x in contraction_dates else 0)

# contraction 국면만 슬라이싱
data11 = data11[data11['contraction'] == 1]
data11

# Selecting relevant columns for the regression
X = data11[['credit03', 'term03', 'equity01']]
y = data11['LQD'] - data11['rf']

# Adding a constant to the independent variables
import statsmodels.api as sm
X = sm.add_constant(X)

# Performing the OLS regression
model11 = sm.OLS(y, X, missing='drop').fit()

# Displaying the regression results
model11.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.226
Model:                            OLS   Adj. R-squared (uncentered):              0.164
Method:                 Least Squares   F-statistic:                              3.607
Date:                Fri, 24 Nov 2023   Prob (F-statistic):                      0.0221
Time:                        22:06:48   Log-Likelihood:                          80.264
No. Observations:                  40   AIC:                                     -154.5
Df Residuals:                      37   BIC:                                     -149.5
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
credit03       4.1284      7.844      0.526      0.602     -11.764      20.021
term03        -3.2026     15.324     -0.209      0.836     -34.252      27.847
equity01       0.2741      0.085      3.221      0.003       0.102       0.446
==============================================================================
Omnibus:                       16.218   Durbin-Watson:                   1.698
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               53.679
Skew:                           0.617   Prob(JB):                     2.21e-12
Kurtosis:                       8.539   Cond. No.                         215.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Contraction - IEF

In [24]:
# 데이터 병합
etf_month_IEF = etf[['DATE', 'IEF']]
data12 = etf_month_IEF.merge(rf, on='DATE', how='inner')
# data12 = data12.merge(credit01, on='DATE', how='inner', suffixes=('', '_credit01'))
# data12 = data12.merge(credit02, on='DATE', how='inner', suffixes=('', '_credit02'))
data12 = data12.merge(credit03, on='DATE', how='inner', suffixes=('', '_credit03'))
# data12 = data12.merge(term01, on='DATE', how='inner', suffixes=('', '_term01'))
# data12 = data12.merge(term02, on='DATE', how='inner', suffixes=('', '_term02'))
data12 = data12.merge(term03, on='DATE', how='inner', suffixes=('', '_term03'))
data12 = data12.merge(equity01, on='DATE', how='inner', suffixes=('', '_equity01'))
# data12 = data12.merge(equity02, on='DATE', how='inner', suffixes=('', '_equity02'))

# Renaming columns as requested
data12.columns = ['DATE', 'IEF', 'rf', 'credit03', 'term03', 'equity01']
data12['DATE'] = pd.to_datetime(data12['DATE'])
data12['DATE'] = data12['DATE'].dt.strftime('%Y-%m')
data12.set_index('DATE', inplace=True)

# Assuming contraction_dates is a list of datetime objects representing the contraction dates
contraction_dates = [pd.to_datetime(date).strftime('%Y-%m') for date in contraction] 
contraction_dates

# contraction 컬럼 추가
data12['contraction'] = data12.index.map(lambda x: 1 if x in contraction_dates else 0)

# contraction 국면만 슬라이싱
data12 = data12[data12['contraction'] == 1]
data12

# Selecting relevant columns for the regression
X = data12[['credit03', 'term03', 'equity01']]
y = data12['IEF'] - data12['rf']

# Adding a constant to the independent variables
import statsmodels.api as sm
X = sm.add_constant(X)

# Performing the OLS regression
model12 = sm.OLS(y, X, missing='drop').fit()

# Displaying the regression results
model12.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.152
Model:                            OLS   Adj. R-squared (uncentered):              0.083
Method:                 Least Squares   F-statistic:                              2.204
Date:                Fri, 24 Nov 2023   Prob (F-statistic):                       0.104
Time:                        22:06:48   Log-Likelihood:                          95.746
No. Observations:                  40   AIC:                                     -185.5
Df Residuals:                      37   BIC:                                     -180.4
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
credit03       4.9594      5.326      0.931      0.358      -5.833      15.752
term03        -5.3174     10.406     -0.511      0.612     -26.402      15.767
equity01      -0.0576      0.058     -0.997      0.325      -0.175       0.059
==============================================================================
Omnibus:                        1.683   Durbin-Watson:                   1.964
Prob(Omnibus):                  0.431   Jarque-Bera (JB):                0.818
Skew:                           0.286   Prob(JB):                        0.664
Kurtosis:                       3.404   Cond. No.                         215.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Buy and Hold - SPY

In [25]:
# 데이터 병합
etf_month_spy = etf[['DATE', 'SPY']]
data13 = etf_month_spy.merge(rf, on='DATE', how='inner')
# data13 = data13.merge(credit01, on='DATE', how='inner', suffixes=('', '_credit01'))
# data13 = data13.merge(credit02, on='DATE', how='inner', suffixes=('', '_credit02'))
data13 = data13.merge(credit03, on='DATE', how='inner', suffixes=('', '_credit03'))
# data13 = data13.merge(term01, on='DATE', how='inner', suffixes=('', '_term01'))
# data13 = data13.merge(term02, on='DATE', how='inner', suffixes=('', '_term02'))
data13 = data13.merge(term03, on='DATE', how='inner', suffixes=('', '_term03'))
data13 = data13.merge(equity01, on='DATE', how='inner', suffixes=('', '_equity01'))
# data13 = data13.merge(equity02, on='DATE', how='inner', suffixes=('', '_equity02'))

# Renaming columns as requested
data13.columns = ['DATE', 'SPY', 'rf', 'credit03', 'term03', 'equity01']
data13['DATE'] = pd.to_datetime(data13['DATE'])
data13['DATE'] = data13['DATE'].dt.strftime('%Y-%m')
data13.set_index('DATE', inplace=True)

# Assuming bh_dates is a list of datetime objects representing the bh dates
bh_dates = [pd.to_datetime(date).strftime('%Y-%m') for date in bh] 
bh_dates

# bh 컬럼 추가
data13['bh'] = data13.index.map(lambda x: 1 if x in bh_dates else 0)

# bh 국면만 슬라이싱
data13 = data13[data13['bh'] == 1]
data13

# Selecting relevant columns for the regression
X = data13[['credit03', 'term03', 'equity01']]
y = data13['SPY'] - data13['rf']

# Adding a constant to the independent variables
import statsmodels.api as sm
X = sm.add_constant(X)

# Performing the OLS regression
model13 = sm.OLS(y, X, missing='drop').fit()

# Displaying the regression results
model13.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.950
Model:                            OLS   Adj. R-squared (uncentered):              0.950
Method:                 Least Squares   F-statistic:                              1420.
Date:                Fri, 24 Nov 2023   Prob (F-statistic):                   1.54e-144
Time:                        22:06:48   Log-Likelihood:                          726.92
No. Observations:                 225   AIC:                                     -1448.
Df Residuals:                     222   BIC:                                     -1438.
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
credit03      -8.8939      1.116     -7.966      0.000     -11.094      -6.694
term03        15.3098      1.926      7.950      0.000      11.515      19.105
equity01       1.0089      0.016     64.600      0.000       0.978       1.040
==============================================================================
Omnibus:                       20.718   Durbin-Watson:                   0.167
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               24.621
Skew:                          -0.805   Prob(JB):                     4.50e-06
Kurtosis:                       2.818   Cond. No.                         143.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Buy and Hold - LQD

In [26]:
# 데이터 병합
etf_month_LQD = etf[['DATE', 'LQD']]
data14 = etf_month_LQD.merge(rf, on='DATE', how='inner')
# data14 = data14.merge(credit01, on='DATE', how='inner', suffixes=('', '_credit01'))
# data14 = data14.merge(credit02, on='DATE', how='inner', suffixes=('', '_credit02'))
data14 = data14.merge(credit03, on='DATE', how='inner', suffixes=('', '_credit03'))
# data14 = data14.merge(term01, on='DATE', how='inner', suffixes=('', '_term01'))
# data14 = data14.merge(term02, on='DATE', how='inner', suffixes=('', '_term02'))
data14 = data14.merge(term03, on='DATE', how='inner', suffixes=('', '_term03'))
data14 = data14.merge(equity01, on='DATE', how='inner', suffixes=('', '_equity01'))
# data14 = data14.merge(equity02, on='DATE', how='inner', suffixes=('', '_equity02'))

# Renaming columns as requested
data14.columns = ['DATE', 'LQD', 'rf', 'credit03', 'term03', 'equity01']
data14['DATE'] = pd.to_datetime(data14['DATE'])
data14['DATE'] = data14['DATE'].dt.strftime('%Y-%m')
data14.set_index('DATE', inplace=True)

# Assuming bh_dates is a list of datetime objects representing the bh dates
bh_dates = [pd.to_datetime(date).strftime('%Y-%m') for date in bh] 
bh_dates

# bh 컬럼 추가
data14['bh'] = data14.index.map(lambda x: 1 if x in bh_dates else 0)

# bh 국면만 슬라이싱
data14 = data14[data14['bh'] == 1]
data14

# Selecting relevant columns for the regression
X = data14[['credit03', 'term03', 'equity01']]
y = data14['LQD'] - data14['rf']

# Adding a constant to the independent variables
import statsmodels.api as sm
X = sm.add_constant(X)

# Performing the OLS regression
model14 = sm.OLS(y, X, missing='drop').fit()

# Displaying the regression results
model14.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.106
Model:                            OLS   Adj. R-squared (uncentered):              0.094
Method:                 Least Squares   F-statistic:                              8.763
Date:                Fri, 24 Nov 2023   Prob (F-statistic):                    1.62e-05
Time:                        22:06:48   Log-Likelihood:                          530.67
No. Observations:                 225   AIC:                                     -1055.
Df Residuals:                     222   BIC:                                     -1045.
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
credit03      -6.6931      2.671     -2.506      0.013     -11.957      -1.430
term03        11.5208      4.607      2.501      0.013       2.442      20.600
equity01       0.1691      0.037      4.526      0.000       0.095       0.243
==============================================================================
Omnibus:                       60.466   Durbin-Watson:                   1.502
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              588.996
Skew:                           0.691   Prob(JB):                    1.26e-128
Kurtosis:                      10.805   Cond. No.                         143.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Buy and Hold - IEF

In [27]:
# 데이터 병합
etf_month_IEF = etf[['DATE', 'IEF']]
data15 = etf_month_IEF.merge(rf, on='DATE', how='inner')
# data15 = data15.merge(credit01, on='DATE', how='inner', suffixes=('', '_credit01'))
# data15 = data15.merge(credit02, on='DATE', how='inner', suffixes=('', '_credit02'))
data15 = data15.merge(credit03, on='DATE', how='inner', suffixes=('', '_credit03'))
# data15 = data15.merge(term01, on='DATE', how='inner', suffixes=('', '_term01'))
# data15 = data15.merge(term02, on='DATE', how='inner', suffixes=('', '_term02'))
data15 = data15.merge(term03, on='DATE', how='inner', suffixes=('', '_term03'))
data15 = data15.merge(equity01, on='DATE', how='inner', suffixes=('', '_equity01'))
# data15 = data15.merge(equity02, on='DATE', how='inner', suffixes=('', '_equity02'))

# Renaming columns as requested
data15.columns = ['DATE', 'IEF', 'rf', 'credit03', 'term03', 'equity01']
data15['DATE'] = pd.to_datetime(data15['DATE'])
data15['DATE'] = data15['DATE'].dt.strftime('%Y-%m')
data15.set_index('DATE', inplace=True)

# Assuming bh_dates is a list of datetime objects representing the bh dates
bh_dates = [pd.to_datetime(date).strftime('%Y-%m') for date in bh] 
bh_dates

# bh 컬럼 추가
data15['bh'] = data15.index.map(lambda x: 1 if x in bh_dates else 0)

# bh 국면만 슬라이싱
data15 = data15[data15['bh'] == 1]
data15

# Selecting relevant columns for the regression
X = data15[['credit03', 'term03', 'equity01']]
y = data15['IEF'] - data15['rf']

# Adding a constant to the independent variables
import statsmodels.api as sm
X = sm.add_constant(X)

# Performing the OLS regression
model15 = sm.OLS(y, X, missing='drop').fit()

# Displaying the regression results
model15.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.077
Model:                            OLS   Adj. R-squared (uncentered):              0.064
Method:                 Least Squares   F-statistic:                              6.155
Date:                Fri, 24 Nov 2023   Prob (F-statistic):                    0.000489
Time:                        22:06:48   Log-Likelihood:                          561.82
No. Observations:                 225   AIC:                                     -1118.
Df Residuals:                     222   BIC:                                     -1107.
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
credit03      -5.8796      2.326     -2.528      0.012     -10.462      -1.297
term03        10.3362      4.011      2.577      0.011       2.431      18.241
equity01      -0.1091      0.033     -3.353      0.001      -0.173      -0.045
==============================================================================
Omnibus:                        2.621   Durbin-Watson:                   1.505
Prob(Omnibus):                  0.270   Jarque-Bera (JB):                2.705
Skew:                           0.034   Prob(JB):                        0.259
Kurtosis:                       3.533   Cond. No.                         143.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### OLS result

In [28]:
co1 = model01.params
co2 = model02.params
co3 = model03.params
co4 = model04.params
co5 = model05.params
co6 = model06.params
co7 = model07.params
co8 = model08.params
co9 = model09.params
co10 = model10.params
co11 = model11.params
co12 = model12.params
co13 = model13.params
co14 = model14.params
co15 = model15.params

df1 = pd.DataFrame(co1, columns=['Coefficient'])
df2 = pd.DataFrame(co2, columns=['Coefficient'])
df3 = pd.DataFrame(co3, columns=['Coefficient'])
df4 = pd.DataFrame(co4, columns=['Coefficient'])
df5 = pd.DataFrame(co5, columns=['Coefficient'])
df6 = pd.DataFrame(co6, columns=['Coefficient'])
df7 = pd.DataFrame(co7, columns=['Coefficient'])
df8 = pd.DataFrame(co8, columns=['Coefficient'])
df9 = pd.DataFrame(co9, columns=['Coefficient'])
df10 = pd.DataFrame(co10, columns=['Coefficient'])
df11 = pd.DataFrame(co11, columns=['Coefficient'])
df12 = pd.DataFrame(co12, columns=['Coefficient'])
df13 = pd.DataFrame(co13, columns=['Coefficient'])
df14 = pd.DataFrame(co14, columns=['Coefficient'])
df15 = pd.DataFrame(co15, columns=['Coefficient'])


ols_result_coef = pd.concat([df1,df4,df7,df10,df13,df2,df5,df8,df11,df14,df3,df6,df9,df12,df15], axis=1)

ols_result_coef.columns = [['recovery-SPY', 'expansion-SPY','slowdown-SPY','contraction-SPY','buy_and_hold-SPY',
                            'recovery-LQD','expansion-LQD','slowdown-LQD','contraction-LQD','buy_and_hold-LQD',
                            'recovery-IEF',  'expansion-IEF','slowdown-IEF', 'contraction-IEF', 'buy_and_hold-IEF']]

In [29]:
ols_result_coef

,recovery-SPY,expansion-SPY,slowdown-SPY,contraction-SPY,buy_and_hold-SPY,recovery-LQD,expansion-LQD,slowdown-LQD,contraction-LQD,buy_and_hold-LQD,recovery-IEF,expansion-IEF,slowdown-IEF,contraction-IEF,buy_and_hold-IEF
credit03,-3.277136,-12.185785,-32.530870,-3.231329,-8.893916,0.836119,-15.997660,-33.598700,4.128373,-6.693099,-2.287824,-15.426920,-27.725375,4.959402,-5.879599
term03,6.515099,19.847681,56.209945,6.842268,15.309797,1.562411,25.002965,54.036901,-3.202646,11.520797,5.548824,24.135302,44.003883,-5.317442,10.336180
equity01,0.991614,1.028213,0.984852,1.011469,1.008935,0.120392,0.099054,-0.025623,0.274089,0.169104,-0.123882,-0.058466,-0.245358,-0.057589,-0.109065


### 시각화 (Panel A)

In [30]:
# data = {
#     'Recovery': [5, 3, 2, 4, 6],  # 회복 국면에서의 수익률(베타계수)
#     'Expansion': [7, 5, 3, 6, 8],  # 확장 국면에서의 수익률(베타계수)
#     'Slowdown': [-2, -1, 0, -1, -2],  # 둔화 국면에서의 수익률(베타계수)
#     'Contraction': [-4, -3, -2, -3, -4],  # 수축 국면에서의 수익률(베타계수)
#     'Buy & Hold': [4, 3, 5, 4, 5]  # 매수 및 보유 국면에서의 수익률(베타계수)
# }
# asset_classes = ['SPY', 'LQD', 'IEF']
# phases = list(data.keys())

# # 데이터를 기반으로 그래프를 그립니다.
# n = len(asset_classes)
# r = np.arange(n)
# width = 0.15

# fig, ax = plt.subplots(figsize=(10, 6))

# for i, phase in enumerate(phases):
#     ax.bar(r + i*width, data[phase], width=width, label=phase)

# # 축 이름과 제목을 추가합니다.
# ax.set_xlabel('Asset Class')
# ax.set_ylabel('Performance (%)')
# ax.set_title('Risk Premia Mapping and Conditional Performance')
# ax.set_xticks(r + width * (len(phases) - 1) / 2)
# ax.set_xticklabels(asset_classes)
# ax.legend()

# # 그래프를 표시합니다.
# plt.show()

In [31]:
ols_result_coef.to_csv('./ols_result_coef.csv')

In [32]:
ols_result_coef

,recovery-SPY,expansion-SPY,slowdown-SPY,contraction-SPY,buy_and_hold-SPY,recovery-LQD,expansion-LQD,slowdown-LQD,contraction-LQD,buy_and_hold-LQD,recovery-IEF,expansion-IEF,slowdown-IEF,contraction-IEF,buy_and_hold-IEF
credit03,-3.277136,-12.185785,-32.530870,-3.231329,-8.893916,0.836119,-15.997660,-33.598700,4.128373,-6.693099,-2.287824,-15.426920,-27.725375,4.959402,-5.879599
term03,6.515099,19.847681,56.209945,6.842268,15.309797,1.562411,25.002965,54.036901,-3.202646,11.520797,5.548824,24.135302,44.003883,-5.317442,10.336180
equity01,0.991614,1.028213,0.984852,1.011469,1.008935,0.120392,0.099054,-0.025623,0.274089,0.169104,-0.123882,-0.058466,-0.245358,-0.057589,-0.109065


In [33]:
def anuual_ret(df):
    etf = (((df.iloc[:,0]) + 1) ** 12) - 1  # etf 연 평균 수익률 -> 소수점 단위 변경
    rf = (((df.iloc[:,1]) + 1) ** 12) - 1  # rf
    credit = (((df.iloc[:,2] ) + 1) ** 12) - 1  # credit
    term = (((df.iloc[:,3] ) + 1) ** 12) - 1  # term
    equity = (((df.iloc[:,4]) + 1) ** 12) - 1  # equity
    lst = [etf.mean(), rf.mean(), credit.mean(), term.mean(), equity.mean()] 
    return lst  # 소수점 단위 값

In [34]:
# def anuual_ret(df):
#     etf = (df.iloc[:,0]).mean()  # etf 연 평균 수익률 -> 소수점 단위 변경
#     rf = (df.iloc[:,1]).mean()  # rf
#     credit = (df.iloc[:,2] ).mean()  # credit
#     term = (df.iloc[:,3] ).mean()# term
#     equity = (df.iloc[:,4]).mean()  # equity
#     lst = [etf, rf, credit, term, equity] 
#     return lst  # 소수점 단위 값

In [35]:
print('ret01 =',anuual_ret(data01))  # Recovery-SPY
print('ret02 =',anuual_ret(data02))  # Recovery-LQD
print('ret03 =',anuual_ret(data03))  # Recovery-IEF
print('ret04 =',anuual_ret(data04))  # Expansion-SPY
print('ret05 =',anuual_ret(data05))  # Expansion-LQD
print('ret06 =',anuual_ret(data06))  # Expansion-IEF
print('ret07 =',anuual_ret(data07))  # Slowdown-SPY
print('ret08 =',anuual_ret(data08))  # Slowdown-LQD
print('ret09 =',anuual_ret(data09))  # Slowdown-IEF
print('ret10 =',anuual_ret(data10))  # Contraction-SPY
print('ret11 =',anuual_ret(data11))  # Contraction-LQD
print('ret12 =',anuual_ret(data12))  # Contraction-IEF
print('ret13 =',anuual_ret(data13))  # Buy and Hold-SPY
print('ret14 =',anuual_ret(data14))  # Buy and Hold-LQD
print('ret15 =',anuual_ret(data15))  # Buy and Hold-IEF

# [etf, rf, credit, term, equity] 

ret01 = [0.47739051940075944, 0.04563651745348951, 0.03806424848649424, 0.020933472915717767, 0.41166760256144397]
ret02 = [0.16304007007373927, 0.04563651745348951, 0.03806424848649424, 0.020933472915717767, 0.41166760256144397]
ret03 = [0.05434076292219162, 0.04563651745348951, 0.03806424848649424, 0.020933472915717767, 0.41166760256144397]
ret04 = [0.2917469025960519, 0.11499050132059661, 0.03254846437839061, 0.018308586907374226, 0.22938108032823804]
ret05 = [0.04781179442972429, 0.11499050132059661, 0.03254846437839061, 0.018308586907374226, 0.22938108032823804]
ret06 = [0.022053784000393303, 0.11499050132059661, 0.03254846437839061, 0.018308586907374226, 0.22938108032823804]
ret07 = [-0.048072104134659704, 0.2335030376043484, 0.023492815414992253, 0.010997448094368431, -0.09297089601407157]
ret08 = [0.0031134475521303654, 0.2335030376043484, 0.023492815414992253, 0.010997448094368431, -0.09297089601407157]
ret09 = [0.07748043226695259, 0.2335030376043484, 0.023492815414992253, 0.

In [36]:
ret01 = anuual_ret(data01)  # Recovery-SPY
ret02 = anuual_ret(data02)  # Recovery-LQD
ret03 = anuual_ret(data03)  # Recovery-IEF
ret04 = anuual_ret(data04)  # Expansion-SPY
ret05 = anuual_ret(data05)  # Expansion-LQD
ret06 = anuual_ret(data06)  # Expansion-IEF
ret07 = anuual_ret(data07)  # Slowdown-SPY
ret08 = anuual_ret(data08)  # Slowdown-LQD
ret09 = anuual_ret(data09)  # Slowdown-IEF
ret10 = anuual_ret(data10)  # Contraction-SPY
ret11 = anuual_ret(data11)  # Contraction-LQD
ret12 = anuual_ret(data12)  # Contraction-IEF
ret13 = anuual_ret(data13)  # Buy and Hold-SPY
ret14 = anuual_ret(data14)  # Buy and Hold-LQD
ret15 = anuual_ret(data15)  # Buy and Hold-IEF

In [37]:
rets = [ret01, ret04, ret07, ret10, ret13, ret02, ret05, ret08, ret11, ret14, ret03, ret06, ret09, ret12, ret15]
annual_return = pd.DataFrame(rets, columns=['etf', 'rf', 'credit', 'term', 'equity'])
annual_return.to_csv('./annual_return.csv')

In [38]:
ols_result_coef

,recovery-SPY,expansion-SPY,slowdown-SPY,contraction-SPY,buy_and_hold-SPY,recovery-LQD,expansion-LQD,slowdown-LQD,contraction-LQD,buy_and_hold-LQD,recovery-IEF,expansion-IEF,slowdown-IEF,contraction-IEF,buy_and_hold-IEF
credit03,-3.277136,-12.185785,-32.530870,-3.231329,-8.893916,0.836119,-15.997660,-33.598700,4.128373,-6.693099,-2.287824,-15.426920,-27.725375,4.959402,-5.879599
term03,6.515099,19.847681,56.209945,6.842268,15.309797,1.562411,25.002965,54.036901,-3.202646,11.520797,5.548824,24.135302,44.003883,-5.317442,10.336180
equity01,0.991614,1.028213,0.984852,1.011469,1.008935,0.120392,0.099054,-0.025623,0.274089,0.169104,-0.123882,-0.058466,-0.245358,-0.057589,-0.109065


In [39]:
ret13

[0.2432068725065515,
 0.10856705972679556,
 0.0328618141046055,
 0.01733961433883553,
 0.18706465791616703]

In [40]:
# y(SPY) 
re_SPY = ols_result_coef.loc['credit03','recovery-SPY']      * ret01[2] + ols_result_coef.loc['term03', 'recovery-SPY']     * ret01[3] + ols_result_coef.loc['equity01', 'recovery-SPY']     * ret01[4]
ex_SPY = ols_result_coef.loc['credit03','expansion-SPY']     * ret04[2] + ols_result_coef.loc['term03', 'expansion-SPY']    * ret04[3] + ols_result_coef.loc['equity01', 'expansion-SPY']    * ret04[4]
sl_SPY = ols_result_coef.loc['credit03','slowdown-SPY']      * ret07[2] + ols_result_coef.loc['term03', 'slowdown-SPY']     * ret07[3] + ols_result_coef.loc['equity01', 'slowdown-SPY']     * ret07[4]
con_SPY =ols_result_coef.loc['credit03','contraction-SPY']   * ret10[2] + ols_result_coef.loc['term03', 'contraction-SPY']  * ret10[3] + ols_result_coef.loc['equity01', 'contraction-SPY']  * ret10[4]
bh_SPY = ols_result_coef.loc['const','buy_and_hold-SPY'] + ols_result_coef.loc['credit03','buy_and_hold-SPY']  * ret13[2] + ols_result_coef.loc['term03', 'buy_and_hold-SPY'] * ret13[3] + ols_result_coef.loc['equity01', 'buy_and_hold-SPY'] * ret13[4]

KeyError: 'const'

In [ ]:
# y(LQD) 
re_LQD =  ols_result_coef.loc['credit03','recovery-LQD']     * ret02[2] + ols_result_coef.loc['term03', 'recovery-LQD']     * ret02[3] + ols_result_coef.loc['equity01', 'recovery-LQD']     * ret02[4]
ex_LQD =  ols_result_coef.loc['credit03','expansion-LQD']    * ret05[2] + ols_result_coef.loc['term03', 'expansion-LQD']    * ret05[3] + ols_result_coef.loc['equity01', 'expansion-LQD']    * ret05[4]
sl_LQD =  ols_result_coef.loc['credit03','slowdown-LQD']     * ret08[2] + ols_result_coef.loc['term03', 'slowdown-LQD']     * ret08[3] + ols_result_coef.loc['equity01', 'slowdown-LQD']     * ret08[4]
con_LQD = ols_result_coef.loc['credit03','contraction-LQD']  * ret11[2] + ols_result_coef.loc['term03', 'contraction-LQD']  * ret11[3] + ols_result_coef.loc['equity01', 'contraction-LQD']  * ret11[4]
bh_LQD =  ols_result_coef.loc['const','buy_and_hold-LQD'] + ols_result_coef.loc['credit03','buy_and_hold-LQD'] * ret14[2] + ols_result_coef.loc['term03', 'buy_and_hold-LQD'] * ret14[3] + ols_result_coef.loc['equity01', 'buy_and_hold-LQD'] * ret14[4]

In [ ]:
# y(IEF) 
re_IEF = ols_result_coef.loc['credit03','recovery-IEF']     * ret03[2] + ols_result_coef.loc['term03', 'recovery-IEF']     * ret03[3] + ols_result_coef.loc['equity01', 'recovery-IEF']     * ret03[4]
ex_IEF =  ols_result_coef.loc['credit03','expansion-IEF']    * ret06[2] + ols_result_coef.loc['term03', 'expansion-IEF']    * ret06[3] + ols_result_coef.loc['equity01', 'expansion-IEF']    * ret06[4]
sl_IEF = ols_result_coef.loc['credit03','slowdown-IEF']     * ret09[2] + ols_result_coef.loc['term03', 'slowdown-IEF']     * ret09[3] + ols_result_coef.loc['equity01', 'slowdown-IEF']     * ret09[4]
con_IEF = ols_result_coef.loc['credit03','contraction-IEF']  * ret12[2] + ols_result_coef.loc['term03', 'contraction-IEF']  * ret12[3] + ols_result_coef.loc['equity01', 'contraction-IEF']  * ret12[4]
bh_IEF =  ols_result_coef.loc['const','buy_and_hold-IEF'] + ols_result_coef.loc['credit03','buy_and_hold-IEF'] * ret15[2] + ols_result_coef.loc['term03', 'buy_and_hold-IEF'] * ret15[3] + ols_result_coef.loc['equity01', 'buy_and_hold-IEF'] * ret15[4]

In [ ]:
print(bh_SPY)
print(bh_LQD)
print(bh_IEF)

buy_and_hold-SPY    0.470247
dtype: float64
buy_and_hold-LQD    0.439586
dtype: float64
buy_and_hold-IEF    0.344164
dtype: float64


In [ ]:
# data = {
#     'SPY' : [re_SPY, ex_SPY, sl_SPY, con_SPY, bh_SPY], # SPY에서의 국면별 수익률
#     'LQD' : [re_LQD, ex_LQD, sl_LQD, con_LQD, bh_LQD], # LQD에서의 국면별 수익률
#     'IEF' : [re_IEF, ex_IEF, sl_IEF, con_IEF, bh_IEF], # IEF에서의 국면별 수익률
# }

# regime_classes = ['recovery', 'expansion', 'slowdown', 'contraction', 'buy & hold']
# phases = list.data(keys())

# # 데이터 기반 시각화
# n = len(regime_classes)
# r = np.arange(n)
# width = 0.15

# fig, ax = plt.subplots(figsize=(10, 6))

# for i, phase in enumerate(phases):
#     ax.bar(r + i*width, data[phase], width=width, label=phase)

# # 축 이름 및 제목 설정
# ax.set_xlabel('Asset Class')
# ax.set_ylabel('Performance (%)')
# ax.set_title('Risk Premia Mapping and Conditional Performance')
# ax.set_xticks(r + width * (len(phases) - 1) / 2)
# ax.set_xticklabels(asset_classes)
# ax.legend()

# plt.show()

In [ ]:
# float(ols_result_coef.loc['rf', 'recovery-SPY'] * ret01[1]) > 0

In [ ]:
# contributions = {
#     'Risk-Free Rate': ols_result_coef.loc['rf', 'recovery-SPY'] * ret01[1],
#     'Credit Premium': ols_result_coef.loc['credit03', 'recovery-SPY'] * ret01[2],
#     'Term Premium': ols_result_coef.loc['term03', 'recovery-SPY'] * ret01[3],
#     'Equity Premium': ols_result_coef.loc['equity01', 'recovery-SPY'] * ret01[4]
# }

# # Plotting the stacked bar chart
# fig, ax = plt.subplots()

# # Initialize the bottom parameters for positive and negative stacks
# bottom_positive = 0
# bottom_negative = 0
# for name, value in contributions.items():
#     if float(value) > 0:
#         # Stack positive values on top of each other
#         ax.bar('recovery-SPY', value, label=name, bottom=bottom_positive)
#         bottom_positive += value
#     else:
#         # Stack negative values below each other
#         ax.bar('recovery-SPY', value, label=name, bottom=bottom_negative)
#         bottom_negative += value  # Since value is negative, this subtracts it

# ax.set_ylabel('Contribution to SPY Recovery Premia')
# ax.legend()

# plt.show()

In [ ]:
# fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharey=True)

# # Titles for each subplot
# titles = ['SPY', 'LQD', 'IEF']

# # Plot each bar chart in a separate subplot with synthetic data
# for i, ax in enumerate(axes.flatten()):
#     ax.bar('recovery-{}'.format(titles[i]), contributions['Risk-Free Rate'], label='Risk-Free Rate', width=0.1, color='green')
#     ax.bar('recovery-{}'.format(titles[i]), contributions['Credit Premium'], label='Credit Premium', width=0.1, color='blue', bottom=contributions['Risk-Free Rate'])
#     ax.bar('recovery-{}'.format(titles[i]), contributions['Term Premium'], label='Term Premium', width=0.1, color='orange', bottom=contributions['Risk-Free Rate'] + contributions['Credit Premium'])
#     ax.bar('recovery-{}'.format(titles[i]), contributions['Equity Premium'], label='Equity Premium', width=0.1, color='red', bottom=contributions['Risk-Free Rate'] + contributions['Credit Premium'] + contributions['Term Premium'])
#     ax.set_title(titles[i])

# # Set common labels
# fig.text(0.5, 0.04, 'Economic Indicator', ha='center', va='center')
# fig.text(0.06, 0.5, 'Contribution to Recovery Premia', ha='center', va='center', rotation='vertical')

# # Display the legend
# axes[0].legend(loc='upper left')
# plt.tight_layout()
# plt.show()